# pyUmbral python API

## Setting the default curve

The first time you use umbral, you may want to specify an elliptic curve to use.  If you do not specify a curve, secp256k1 will be used for all operations, with a slight performace hit for the lookup.

To set the default curve use `umbral.config.set_default_curve()`

Note: you can only set the dafault once, or `UmbralConfigurationError` will be raised.

In [1]:
from umbral.config import set_default_curve
from umbral.curve import SECP256K1

set_default_curve(SECP256K1)


## Generate Umbral Keys

In [2]:
from umbral import keys


# Alice's Keys
alice_private_key = keys.UmbralPrivateKey.gen_key()
alice_public_key = alice_private_key.get_pubkey()

# Bob's Keys
bob_private_key = keys.UmbralPrivateKey.gen_key()
bob_public_key = bob_private_key.get_pubkey()


## Alice encrypts data; Generates a capsule

In [3]:
from umbral import umbral


plaintext = b'attack at dawn'
ciphertext, capsule = umbral.encrypt(alice_public_key, plaintext)


## Alice decrypts data for self

In [4]:
cleartext = umbral.decrypt(capsule, alice_private_key, ciphertext)
print(cleartext)


b'attack at dawn'


# Proxy Re-encryption

![proxy_reencryption](https://cdn-images-1.medium.com/max/1200/0*yTKUeeuKPu-aIZdw.)

## Split Rekeys

In [5]:
N, M = 1, 1
kfrags, vkeys = umbral.split_rekey(alice_private_key, bob_public_key, M, N)



## Ursula Re-encrypts; Bob attaches fragments to capsule


In [6]:
for kfrag in kfrags:
    cfrag = umbral.reencrypt(kfrag, capsule)
    capsule.attach_cfrag(cfrag)


## Bob activates and opens the capsule; Decrypts data from Alice.

In [7]:
reencrypted_cleartext = umbral.decrypt(capsule, bob_private_key, ciphertext, alice_public_key)
print(reencrypted_cleartext)


b'attack at dawn'
